In [1]:
import os
import torch
from torch import nn
from torch import optim
from torch.utils.data import random_split, DataLoader
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import random

## Check and setup GPU device

In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Define sample

In [4]:
sample = 2

## Define model

In [5]:
def Network(out=960):
    return nn.Sequential(
            nn.Linear(4, 300),
            nn.ReLU(),
            nn.Linear(300, 300),
            nn.ReLU(),
            nn.Linear(300, 300),
            nn.ReLU(),
            nn.Linear(300, 300),
            nn.ReLU(),
            nn.Linear(300, 300),
            nn.ReLU(),
            nn.Linear(300, 300),
            nn.ReLU(),
            nn.Linear(300, out)
        ).to(device)

## Define loss

In [6]:
loss = nn.CrossEntropyLoss()

# Euler

In [7]:
ontology = "chi"

In [8]:
def euler_to_index(x):
    return 0.5*x + 480

## Get data

In [9]:
polys = torch.Tensor(pd.read_csv(f"{sample}/polys.csv").values)
labels = pd.read_csv(f"{sample}/labels.csv")[ontology].values
labels = torch.Tensor(list((map(euler_to_index, labels)))).to(torch.int64)

In [10]:
data = [(polys[i], labels[i]) for i in range(len(polys))]
random.shuffle(data)

## Cross-validation data and dataloaders

In [11]:
k = 5
kf = KFold(n_splits=k)
kf.get_n_splits([[1,2],[3,4]])

kfold_data = list()

for train_idx, val_idx in kf.split(data):
    train = [data[i] for i in train_idx]
    val = [data[i] for i in val_idx]
    kfold_data.append((train, val))

In [12]:
kfold_loaders = list()
for train, val in kfold_data:
    train_loader = DataLoader(train, batch_size=1000)
    val_loader = DataLoader(val, batch_size=1000)
    kfold_loaders.append((train_loader, val_loader))

## Training and validation loops

In [ ]:
epochs = 250
for k, (train_loader, val_loader) in enumerate(kfold_loaders):
    if k == 0:
        continue
    model = Network()
    params = model.parameters()
    opt = optim.Adam(params)
    best_acc = 0.0
    for epoch in range(epochs):
        losses = list()
        accuracies = list()
        for batch in train_loader:
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            
            # 1: forward
            logit = model(x)

            # 2: compute the objective function
            J = loss(logit, y)

            # 3: cleaning the gradients
            model.zero_grad()
            # params.grad.zero_()

            # 4: accumulate the partial derivatives of J wrt params
            J.backward()
            # params.grad.add_(dJ/dparams)

            # 5: step in the opposite direction of the gradient (update!)
            opt.step()
            # with torch.no_grad(): 
            #    params .= params .- eta * params.grad

            losses.append(J.item())
            accuracies.append(y.eq(logit.detach().argmax(dim=1)).float().mean())


        loss_mean = torch.tensor(losses).mean()
        acc_mean = torch.tensor(accuracies).mean()

        print(f'Epoch {epoch+1}, train loss: {loss_mean:.2f}, train accuracy: {acc_mean:.2f}')

        losses = list()
        accuracies = list()
        for batch in val_loader:
            x, y = batch
            x = x.to(device)
            y = y.to(device)

            # 1: forward
            with torch.no_grad():
                logit = model(x)

            # 2: compute the objective function
            J = loss(logit, y)

            losses.append(J.item())
            accuracies.append(y.eq(logit.detach().argmax(dim=1)).float().mean())

        loss_mean = torch.tensor(losses).mean()
        acc_mean = torch.tensor(accuracies).mean()

        if acc_mean > best_acc:
            best_acc = acc_mean
            PATH = f'{sample}/kfold_training/{k+1}/{ontology}'
            if not os.path.isdir(PATH):
                os.makedirs(PATH)
            torch.save(model, f'{PATH}/best_model_{best_acc:.4f}.pt')

        print(f'Epoch {epoch+1}, val loss: {loss_mean:.2f}, test accuracy: {acc_mean:.2f}')

    print(f'best accuracy obtained: {best_acc}')
    # end of epoch

Epoch 1, train loss: 2.82, train accuracy: 0.16
Epoch 1, val loss: 2.51, test accuracy: 0.21
Epoch 2, train loss: 2.49, train accuracy: 0.20
Epoch 2, val loss: 2.46, test accuracy: 0.20
Epoch 3, train loss: 2.42, train accuracy: 0.22
Epoch 3, val loss: 2.41, test accuracy: 0.21
Epoch 4, train loss: 2.37, train accuracy: 0.22
Epoch 4, val loss: 2.33, test accuracy: 0.22
Epoch 5, train loss: 2.35, train accuracy: 0.23
Epoch 5, val loss: 2.35, test accuracy: 0.22
Epoch 6, train loss: 2.31, train accuracy: 0.23
Epoch 6, val loss: 2.27, test accuracy: 0.25
Epoch 7, train loss: 2.26, train accuracy: 0.24
Epoch 7, val loss: 2.21, test accuracy: 0.26
Epoch 8, train loss: 2.23, train accuracy: 0.24
Epoch 8, val loss: 2.35, test accuracy: 0.20
Epoch 9, train loss: 2.20, train accuracy: 0.25
Epoch 9, val loss: 2.45, test accuracy: 0.15
Epoch 10, train loss: 2.13, train accuracy: 0.26
Epoch 10, val loss: 2.12, test accuracy: 0.29
Epoch 11, train loss: 2.12, train accuracy: 0.27
Epoch 11, val loss:

Epoch 87, val loss: 1.36, test accuracy: 0.39
Epoch 88, train loss: 1.38, train accuracy: 0.54
Epoch 88, val loss: 1.36, test accuracy: 0.43
Epoch 89, train loss: 1.27, train accuracy: 0.54
Epoch 89, val loss: 1.04, test accuracy: 0.58
Epoch 90, train loss: 1.18, train accuracy: 0.58
Epoch 90, val loss: 0.86, test accuracy: 0.71
Epoch 91, train loss: 1.34, train accuracy: 0.56
Epoch 91, val loss: 1.12, test accuracy: 0.67
Epoch 92, train loss: 1.17, train accuracy: 0.58
Epoch 92, val loss: 1.46, test accuracy: 0.32
Epoch 93, train loss: 1.20, train accuracy: 0.59
Epoch 93, val loss: 1.29, test accuracy: 0.38
Epoch 94, train loss: 1.24, train accuracy: 0.59
Epoch 94, val loss: 1.11, test accuracy: 0.54
Epoch 95, train loss: 1.40, train accuracy: 0.54
Epoch 95, val loss: 1.28, test accuracy: 0.45
Epoch 96, train loss: 1.36, train accuracy: 0.54
Epoch 96, val loss: 1.19, test accuracy: 0.54
Epoch 97, train loss: 1.33, train accuracy: 0.55
Epoch 97, val loss: 1.59, test accuracy: 0.40
Epoc

Epoch 172, val loss: 2.07, test accuracy: 0.31
Epoch 173, train loss: 1.05, train accuracy: 0.66
Epoch 173, val loss: 0.96, test accuracy: 0.62
Epoch 174, train loss: 1.22, train accuracy: 0.62
Epoch 174, val loss: 1.26, test accuracy: 0.34
Epoch 175, train loss: 1.17, train accuracy: 0.62
Epoch 175, val loss: 0.75, test accuracy: 0.76
Epoch 176, train loss: 1.16, train accuracy: 0.65
Epoch 176, val loss: 0.60, test accuracy: 0.86
Epoch 177, train loss: 1.21, train accuracy: 0.60
Epoch 177, val loss: 1.65, test accuracy: 0.33
Epoch 178, train loss: 1.10, train accuracy: 0.64
Epoch 178, val loss: 1.72, test accuracy: 0.33
Epoch 179, train loss: 1.15, train accuracy: 0.64
Epoch 179, val loss: 3.75, test accuracy: 0.01
Epoch 180, train loss: 1.14, train accuracy: 0.63
Epoch 180, val loss: 0.56, test accuracy: 0.89
Epoch 181, train loss: 1.21, train accuracy: 0.60
Epoch 181, val loss: 1.00, test accuracy: 0.66
Epoch 182, train loss: 1.19, train accuracy: 0.63
Epoch 182, val loss: 1.93, tes

Epoch 7, train loss: 2.28, train accuracy: 0.24
Epoch 7, val loss: 2.16, test accuracy: 0.28
Epoch 8, train loss: 2.19, train accuracy: 0.25
Epoch 8, val loss: 2.00, test accuracy: 0.30
Epoch 9, train loss: 2.14, train accuracy: 0.26
Epoch 9, val loss: 2.01, test accuracy: 0.28
Epoch 10, train loss: 2.10, train accuracy: 0.27
Epoch 10, val loss: 2.01, test accuracy: 0.31
Epoch 11, train loss: 2.09, train accuracy: 0.28
Epoch 11, val loss: 1.78, test accuracy: 0.36
Epoch 12, train loss: 2.00, train accuracy: 0.28
Epoch 12, val loss: 1.88, test accuracy: 0.29
Epoch 13, train loss: 1.93, train accuracy: 0.30
Epoch 13, val loss: 1.77, test accuracy: 0.32
Epoch 14, train loss: 1.95, train accuracy: 0.31
Epoch 14, val loss: 1.87, test accuracy: 0.27
Epoch 15, train loss: 1.95, train accuracy: 0.31
Epoch 15, val loss: 3.63, test accuracy: 0.02
Epoch 16, train loss: 1.91, train accuracy: 0.32
Epoch 16, val loss: 1.73, test accuracy: 0.33
Epoch 17, train loss: 2.01, train accuracy: 0.31
Epoch 1

Epoch 93, val loss: 1.57, test accuracy: 0.31
Epoch 94, train loss: 1.28, train accuracy: 0.55
Epoch 94, val loss: 2.16, test accuracy: 0.10
Epoch 95, train loss: 1.34, train accuracy: 0.54
Epoch 95, val loss: 0.77, test accuracy: 0.76
Epoch 96, train loss: 1.37, train accuracy: 0.56
Epoch 96, val loss: 0.69, test accuracy: 0.84
Epoch 97, train loss: 1.38, train accuracy: 0.53
Epoch 97, val loss: 1.31, test accuracy: 0.48
Epoch 98, train loss: 1.33, train accuracy: 0.56
Epoch 98, val loss: 2.83, test accuracy: 0.12
Epoch 99, train loss: 1.43, train accuracy: 0.53
Epoch 99, val loss: 0.87, test accuracy: 0.75
Epoch 100, train loss: 1.35, train accuracy: 0.53
Epoch 100, val loss: 0.88, test accuracy: 0.77
Epoch 101, train loss: 1.37, train accuracy: 0.54
Epoch 101, val loss: 1.47, test accuracy: 0.41
Epoch 102, train loss: 1.39, train accuracy: 0.54
Epoch 102, val loss: 1.03, test accuracy: 0.64
Epoch 103, train loss: 1.40, train accuracy: 0.53
Epoch 103, val loss: 2.13, test accuracy: 0

Epoch 178, val loss: 1.90, test accuracy: 0.27
Epoch 179, train loss: 1.05, train accuracy: 0.64
Epoch 179, val loss: 0.88, test accuracy: 0.64
Epoch 180, train loss: 1.13, train accuracy: 0.63
Epoch 180, val loss: 1.19, test accuracy: 0.51
Epoch 181, train loss: 1.07, train accuracy: 0.64
Epoch 181, val loss: 0.49, test accuracy: 0.91
Epoch 182, train loss: 1.05, train accuracy: 0.65
Epoch 182, val loss: 0.86, test accuracy: 0.69
Epoch 183, train loss: 1.15, train accuracy: 0.63
Epoch 183, val loss: 0.98, test accuracy: 0.67
Epoch 184, train loss: 1.01, train accuracy: 0.67
Epoch 184, val loss: 0.62, test accuracy: 0.81
Epoch 185, train loss: 1.09, train accuracy: 0.63
Epoch 185, val loss: 0.57, test accuracy: 0.88
Epoch 186, train loss: 1.08, train accuracy: 0.65
Epoch 186, val loss: 0.99, test accuracy: 0.58
Epoch 187, train loss: 1.13, train accuracy: 0.62
Epoch 187, val loss: 0.58, test accuracy: 0.89
Epoch 188, train loss: 1.03, train accuracy: 0.65
Epoch 188, val loss: 0.50, tes

Epoch 13, train loss: 2.17, train accuracy: 0.27
Epoch 13, val loss: 1.77, test accuracy: 0.36
Epoch 14, train loss: 2.19, train accuracy: 0.27
Epoch 14, val loss: 2.30, test accuracy: 0.23
Epoch 15, train loss: 2.17, train accuracy: 0.27
Epoch 15, val loss: 2.14, test accuracy: 0.27
Epoch 16, train loss: 2.12, train accuracy: 0.28
Epoch 16, val loss: 1.87, test accuracy: 0.30
Epoch 17, train loss: 2.06, train accuracy: 0.29
Epoch 17, val loss: 1.76, test accuracy: 0.37
Epoch 18, train loss: 2.03, train accuracy: 0.29
Epoch 18, val loss: 1.52, test accuracy: 0.49
Epoch 19, train loss: 1.99, train accuracy: 0.31
Epoch 19, val loss: 1.76, test accuracy: 0.39
Epoch 20, train loss: 1.94, train accuracy: 0.32
Epoch 20, val loss: 2.63, test accuracy: 0.05
Epoch 21, train loss: 1.81, train accuracy: 0.34
Epoch 21, val loss: 1.48, test accuracy: 0.46
Epoch 22, train loss: 1.68, train accuracy: 0.38
Epoch 22, val loss: 1.27, test accuracy: 0.57
Epoch 23, train loss: 1.60, train accuracy: 0.40
E

Epoch 99, val loss: 0.97, test accuracy: 0.68


# h11

In [19]:
ontology = "h11"

## Get data

In [20]:
polys = torch.Tensor(pd.read_csv(f"{sample}/polys.csv").values)
labels = pd.read_csv(f"{sample}/labels.csv")[ontology].values
labels = torch.Tensor(labels).to(torch.int64)

In [21]:
data = [(polys[i], labels[i]) for i in range(len(polys))]
random.shuffle(data)

## Cross-validation

In [22]:
k = 5
kf = KFold(n_splits=k)
kf.get_n_splits([[1,2],[3,4]])

kfold_data = list()

for train_idx, val_idx in kf.split(data):
    train = [data[i] for i in train_idx]
    val = [data[i] for i in val_idx]
    kfold_data.append((train, val))

In [23]:
kfold_loaders = list()
for train, val in kfold_data:
    train_loader = DataLoader(train, batch_size=1000)
    val_loader = DataLoader(val, batch_size=1000)
    kfold_loaders.append((train_loader, val_loader))

## Training

In [24]:
epochs = 250
for k, (train_loader, val_loader) in enumerate(kfold_loaders):
    model = Network(out=480)
    params = model.parameters()
    opt = optim.Adam(params)
    best_acc = 0.0
    for epoch in range(epochs):
        losses = list()
        accuracies = list()
        for batch in train_loader:
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            
            # 1: forward
            logit = model(x)

            # 2: compute the objective function
            J = loss(logit, y)

            # 3: cleaning the gradients
            model.zero_grad()
            # params.grad.zero_()

            # 4: accumulate the partial derivatives of J wrt params
            J.backward()
            # params.grad.add_(dJ/dparams)

            # 5: step in the opposite direction of the gradient (update!)
            opt.step()
            # with torch.no_grad(): 
            #    params .= params .- eta * params.grad

            losses.append(J.item())
            accuracies.append(y.eq(logit.detach().argmax(dim=1)).float().mean())


        loss_mean = torch.tensor(losses).mean()
        acc_mean = torch.tensor(accuracies).mean()

        print(f'Epoch {epoch+1}, train loss: {loss_mean:.2f}, train accuracy: {acc_mean:.2f}')

        losses = list()
        accuracies = list()
        for batch in val_loader:
            x, y = batch
            x = x.to(device)
            y = y.to(device)

            # 1: forward
            with torch.no_grad():
                logit = model(x)

            # 2: compute the objective function
            J = loss(logit, y)

            losses.append(J.item())
            accuracies.append(y.eq(logit.detach().argmax(dim=1)).float().mean())

        loss_mean = torch.tensor(losses).mean()
        acc_mean = torch.tensor(accuracies).mean()

        if acc_mean > best_acc:
            best_acc = acc_mean
            PATH = f'{sample}/kfold_training/{k+1}/{ontology}'
            if not os.path.isdir(PATH):
                os.makedirs(PATH)
            torch.save(model, f'{PATH}/best_model_{best_acc:.4f}.pt')

        print(f'Epoch {epoch+1}, val loss: {loss_mean:.2f}, test accuracy: {acc_mean:.2f}')

    print(f'best accuracy obtained: {best_acc}')
    # end of epoch

Epoch 1, train loss: 3.17, train accuracy: 0.24
Epoch 1, val loss: 2.40, test accuracy: 0.31
Epoch 2, train loss: 2.24, train accuracy: 0.35
Epoch 2, val loss: 2.12, test accuracy: 0.38
Epoch 3, train loss: 2.07, train accuracy: 0.37
Epoch 3, val loss: 2.00, test accuracy: 0.38
Epoch 4, train loss: 1.96, train accuracy: 0.38
Epoch 4, val loss: 1.90, test accuracy: 0.39
Epoch 5, train loss: 1.86, train accuracy: 0.40
Epoch 5, val loss: 1.81, test accuracy: 0.38
Epoch 6, train loss: 1.79, train accuracy: 0.41
Epoch 6, val loss: 1.77, test accuracy: 0.40
Epoch 7, train loss: 1.73, train accuracy: 0.42
Epoch 7, val loss: 1.71, test accuracy: 0.42
Epoch 8, train loss: 1.68, train accuracy: 0.43
Epoch 8, val loss: 1.65, test accuracy: 0.44
Epoch 9, train loss: 1.63, train accuracy: 0.44
Epoch 9, val loss: 1.57, test accuracy: 0.47
Epoch 10, train loss: 1.61, train accuracy: 0.45
Epoch 10, val loss: 1.63, test accuracy: 0.43
Epoch 11, train loss: 1.57, train accuracy: 0.45
Epoch 11, val loss:

Epoch 87, val loss: 0.93, test accuracy: 0.64
Epoch 88, train loss: 1.08, train accuracy: 0.59
Epoch 88, val loss: 0.99, test accuracy: 0.61
Epoch 89, train loss: 1.04, train accuracy: 0.60
Epoch 89, val loss: 1.48, test accuracy: 0.49
Epoch 90, train loss: 1.04, train accuracy: 0.60
Epoch 90, val loss: 1.31, test accuracy: 0.52
Epoch 91, train loss: 1.07, train accuracy: 0.59
Epoch 91, val loss: 0.99, test accuracy: 0.62
Epoch 92, train loss: 1.05, train accuracy: 0.60
Epoch 92, val loss: 1.07, test accuracy: 0.59
Epoch 93, train loss: 1.02, train accuracy: 0.60
Epoch 93, val loss: 0.95, test accuracy: 0.63
Epoch 94, train loss: 1.04, train accuracy: 0.60
Epoch 94, val loss: 0.97, test accuracy: 0.61
Epoch 95, train loss: 1.02, train accuracy: 0.60
Epoch 95, val loss: 1.00, test accuracy: 0.61
Epoch 96, train loss: 1.02, train accuracy: 0.60
Epoch 96, val loss: 0.91, test accuracy: 0.64
Epoch 97, train loss: 1.03, train accuracy: 0.60
Epoch 97, val loss: 1.03, test accuracy: 0.61
Epoc

Epoch 172, val loss: 0.91, test accuracy: 0.65
Epoch 173, train loss: 0.93, train accuracy: 0.64
Epoch 173, val loss: 0.81, test accuracy: 0.67
Epoch 174, train loss: 0.94, train accuracy: 0.63
Epoch 174, val loss: 0.81, test accuracy: 0.68
Epoch 175, train loss: 0.93, train accuracy: 0.63
Epoch 175, val loss: 1.07, test accuracy: 0.59
Epoch 176, train loss: 0.92, train accuracy: 0.64
Epoch 176, val loss: 0.84, test accuracy: 0.66
Epoch 177, train loss: 0.94, train accuracy: 0.63
Epoch 177, val loss: 1.65, test accuracy: 0.47
Epoch 178, train loss: 0.95, train accuracy: 0.63
Epoch 178, val loss: 0.80, test accuracy: 0.68
Epoch 179, train loss: 0.93, train accuracy: 0.63
Epoch 179, val loss: 0.95, test accuracy: 0.61
Epoch 180, train loss: 0.92, train accuracy: 0.64
Epoch 180, val loss: 0.87, test accuracy: 0.66
Epoch 181, train loss: 0.91, train accuracy: 0.64
Epoch 181, val loss: 0.84, test accuracy: 0.65
Epoch 182, train loss: 0.94, train accuracy: 0.63
Epoch 182, val loss: 0.80, tes

Epoch 7, train loss: 1.63, train accuracy: 0.43
Epoch 7, val loss: 1.80, test accuracy: 0.38
Epoch 8, train loss: 1.60, train accuracy: 0.43
Epoch 8, val loss: 1.52, test accuracy: 0.46
Epoch 9, train loss: 1.54, train accuracy: 0.45
Epoch 9, val loss: 1.48, test accuracy: 0.47
Epoch 10, train loss: 1.50, train accuracy: 0.47
Epoch 10, val loss: 1.53, test accuracy: 0.45
Epoch 11, train loss: 1.45, train accuracy: 0.48
Epoch 11, val loss: 1.44, test accuracy: 0.47
Epoch 12, train loss: 1.44, train accuracy: 0.49
Epoch 12, val loss: 1.38, test accuracy: 0.52
Epoch 13, train loss: 1.41, train accuracy: 0.50
Epoch 13, val loss: 1.34, test accuracy: 0.53
Epoch 14, train loss: 1.37, train accuracy: 0.51
Epoch 14, val loss: 1.35, test accuracy: 0.51
Epoch 15, train loss: 1.36, train accuracy: 0.51
Epoch 15, val loss: 1.39, test accuracy: 0.48
Epoch 16, train loss: 1.33, train accuracy: 0.52
Epoch 16, val loss: 1.29, test accuracy: 0.52
Epoch 17, train loss: 1.59, train accuracy: 0.44
Epoch 1

Epoch 93, val loss: 0.92, test accuracy: 0.64
Epoch 94, train loss: 1.04, train accuracy: 0.60
Epoch 94, val loss: 0.99, test accuracy: 0.62
Epoch 95, train loss: 1.01, train accuracy: 0.61
Epoch 95, val loss: 0.87, test accuracy: 0.66
Epoch 96, train loss: 1.01, train accuracy: 0.61
Epoch 96, val loss: 0.86, test accuracy: 0.66
Epoch 97, train loss: 1.05, train accuracy: 0.60
Epoch 97, val loss: 1.00, test accuracy: 0.61
Epoch 98, train loss: 1.06, train accuracy: 0.61
Epoch 98, val loss: 0.91, test accuracy: 0.66
Epoch 99, train loss: 1.01, train accuracy: 0.61
Epoch 99, val loss: 0.91, test accuracy: 0.65
Epoch 100, train loss: 1.06, train accuracy: 0.60
Epoch 100, val loss: 0.88, test accuracy: 0.65
Epoch 101, train loss: 1.03, train accuracy: 0.61
Epoch 101, val loss: 1.00, test accuracy: 0.59
Epoch 102, train loss: 1.01, train accuracy: 0.61
Epoch 102, val loss: 1.03, test accuracy: 0.60
Epoch 103, train loss: 1.01, train accuracy: 0.61
Epoch 103, val loss: 0.90, test accuracy: 0

...

Epoch 105, val loss: 1.04, test accuracy: 0.60
Epoch 106, train loss: 1.05, train accuracy: 0.60
Epoch 106, val loss: 1.05, test accuracy: 0.60
Epoch 107, train loss: 1.09, train accuracy: 0.59
Epoch 107, val loss: 1.06, test accuracy: 0.58
Epoch 108, train loss: 1.02, train accuracy: 0.60
Epoch 108, val loss: 0.87, test accuracy: 0.66
Epoch 109, train loss: 1.05, train accuracy: 0.59
Epoch 109, val loss: 0.90, test accuracy: 0.65
Epoch 110, train loss: 1.06, train accuracy: 0.60
Epoch 110, val loss: 0.99, test accuracy: 0.61
Epoch 111, train loss: 1.03, train accuracy: 0.60
Epoch 111, val loss: 0.98, test accuracy: 0.62
Epoch 112, train loss: 1.04, train accuracy: 0.60
Epoch 112, val loss: 0.96, test accuracy: 0.63
Epoch 113, train loss: 1.00, train accuracy: 0.61
Epoch 113, val loss: 0.90, test accuracy: 0.64
Epoch 114, train loss: 1.04, train accuracy: 0.60
Epoch 114, val loss: 1.08, test accuracy: 0.58
Epoch 115, train loss: 1.07, train accuracy: 0.60
Epoch 115, val loss: 0.92, tes

Epoch 190, train loss: 0.94, train accuracy: 0.63
Epoch 190, val loss: 0.84, test accuracy: 0.67
Epoch 191, train loss: 0.93, train accuracy: 0.63
Epoch 191, val loss: 0.85, test accuracy: 0.66
Epoch 192, train loss: 0.90, train accuracy: 0.64
Epoch 192, val loss: 0.80, test accuracy: 0.68
Epoch 193, train loss: 0.93, train accuracy: 0.63
Epoch 193, val loss: 0.77, test accuracy: 0.69
Epoch 194, train loss: 0.92, train accuracy: 0.64
Epoch 194, val loss: 1.51, test accuracy: 0.47
Epoch 195, train loss: 0.96, train accuracy: 0.63
Epoch 195, val loss: 0.86, test accuracy: 0.66
Epoch 196, train loss: 0.95, train accuracy: 0.63
Epoch 196, val loss: 0.78, test accuracy: 0.70
Epoch 197, train loss: 0.95, train accuracy: 0.63
Epoch 197, val loss: 0.93, test accuracy: 0.63
Epoch 198, train loss: 0.92, train accuracy: 0.64
Epoch 198, val loss: 0.77, test accuracy: 0.68
Epoch 199, train loss: 0.90, train accuracy: 0.64
Epoch 199, val loss: 1.12, test accuracy: 0.57
Epoch 200, train loss: 1.00, t

Epoch 25, train loss: 1.40, train accuracy: 0.50
Epoch 25, val loss: 1.37, test accuracy: 0.51
Epoch 26, train loss: 1.42, train accuracy: 0.50
Epoch 26, val loss: 1.27, test accuracy: 0.54
Epoch 27, train loss: 1.35, train accuracy: 0.50
Epoch 27, val loss: 1.31, test accuracy: 0.52
Epoch 28, train loss: 1.33, train accuracy: 0.51
Epoch 28, val loss: 1.28, test accuracy: 0.52
Epoch 29, train loss: 1.31, train accuracy: 0.52
Epoch 29, val loss: 1.42, test accuracy: 0.50
Epoch 30, train loss: 1.30, train accuracy: 0.52
Epoch 30, val loss: 1.15, test accuracy: 0.57
Epoch 31, train loss: 1.29, train accuracy: 0.52
Epoch 31, val loss: 1.18, test accuracy: 0.56
Epoch 32, train loss: 1.28, train accuracy: 0.53
Epoch 32, val loss: 1.17, test accuracy: 0.56
Epoch 33, train loss: 1.27, train accuracy: 0.53
Epoch 33, val loss: 1.27, test accuracy: 0.52
Epoch 34, train loss: 1.27, train accuracy: 0.53
Epoch 34, val loss: 1.10, test accuracy: 0.59
Epoch 35, train loss: 1.26, train accuracy: 0.53
E

Epoch 111, val loss: 0.88, test accuracy: 0.65
Epoch 112, train loss: 1.04, train accuracy: 0.61
Epoch 112, val loss: 0.84, test accuracy: 0.67
Epoch 113, train loss: 1.01, train accuracy: 0.62
Epoch 113, val loss: 0.86, test accuracy: 0.66
Epoch 114, train loss: 1.01, train accuracy: 0.62
Epoch 114, val loss: 0.86, test accuracy: 0.66
Epoch 115, train loss: 1.01, train accuracy: 0.61
Epoch 115, val loss: 1.00, test accuracy: 0.61
Epoch 116, train loss: 1.00, train accuracy: 0.62
Epoch 116, val loss: 0.84, test accuracy: 0.67
Epoch 117, train loss: 0.96, train accuracy: 0.63
Epoch 117, val loss: 0.99, test accuracy: 0.61
Epoch 118, train loss: 1.00, train accuracy: 0.62
Epoch 118, val loss: 0.90, test accuracy: 0.65
Epoch 119, train loss: 0.99, train accuracy: 0.62
Epoch 119, val loss: 0.90, test accuracy: 0.63
Epoch 120, train loss: 1.03, train accuracy: 0.61
Epoch 120, val loss: 0.87, test accuracy: 0.66
Epoch 121, train loss: 0.97, train accuracy: 0.62
Epoch 121, val loss: 1.00, tes

Epoch 196, train loss: 0.90, train accuracy: 0.65
Epoch 196, val loss: 0.79, test accuracy: 0.68
Epoch 197, train loss: 0.90, train accuracy: 0.65
Epoch 197, val loss: 0.84, test accuracy: 0.64
Epoch 198, train loss: 0.88, train accuracy: 0.65
Epoch 198, val loss: 0.86, test accuracy: 0.64
Epoch 199, train loss: 0.91, train accuracy: 0.64
Epoch 199, val loss: 0.80, test accuracy: 0.69
Epoch 200, train loss: 0.91, train accuracy: 0.64
Epoch 200, val loss: 0.85, test accuracy: 0.66
Epoch 201, train loss: 0.92, train accuracy: 0.64
Epoch 201, val loss: 1.17, test accuracy: 0.53
Epoch 202, train loss: 0.88, train accuracy: 0.65
Epoch 202, val loss: 0.78, test accuracy: 0.68
Epoch 203, train loss: 0.85, train accuracy: 0.66
Epoch 203, val loss: 1.06, test accuracy: 0.57
Epoch 204, train loss: 0.93, train accuracy: 0.64
Epoch 204, val loss: 0.75, test accuracy: 0.70
Epoch 205, train loss: 0.91, train accuracy: 0.64
Epoch 205, val loss: 0.76, test accuracy: 0.69
Epoch 206, train loss: 0.91, t

## h21

In [27]:
ontology="h21"

## Get data

In [28]:
polys = torch.Tensor(pd.read_csv(f"{sample}/polys.csv").values)
labels = pd.read_csv(f"{sample}/labels.csv")[ontology].values
labels = torch.Tensor(labels).to(torch.int64)

In [29]:
data = [(polys[i], labels[i]) for i in range(len(polys))]
random.shuffle(data)

## Cross-validation

In [30]:
k = 5
kf = KFold(n_splits=k)
kf.get_n_splits([[1,2],[3,4]])

kfold_data = list()

for train_idx, val_idx in kf.split(data):
    train = [data[i] for i in train_idx]
    val = [data[i] for i in val_idx]
    kfold_data.append((train, val))

In [31]:
kfold_loaders = list()
for train, val in kfold_data:
    train_loader = DataLoader(train, batch_size=1000)
    val_loader = DataLoader(val, batch_size=1000)
    kfold_loaders.append((train_loader, val_loader))

## Training

In [32]:
epochs = 250
for k, (train_loader, val_loader) in enumerate(kfold_loaders):
    model = Network(out=480)
    params = model.parameters()
    opt = optim.Adam(params)
    best_acc = 0.0
    for epoch in range(epochs):
        losses = list()
        accuracies = list()
        for batch in train_loader:
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            
            # 1: forward
            logit = model(x)

            # 2: compute the objective function
            J = loss(logit, y)

            # 3: cleaning the gradients
            model.zero_grad()
            # params.grad.zero_()

            # 4: accumulate the partial derivatives of J wrt params
            J.backward()
            # params.grad.add_(dJ/dparams)

            # 5: step in the opposite direction of the gradient (update!)
            opt.step()
            # with torch.no_grad(): 
            #    params .= params .- eta * params.grad

            losses.append(J.item())
            accuracies.append(y.eq(logit.detach().argmax(dim=1)).float().mean())


        loss_mean = torch.tensor(losses).mean()
        acc_mean = torch.tensor(accuracies).mean()

        print(f'Epoch {epoch+1}, train loss: {loss_mean:.2f}, train accuracy: {acc_mean:.2f}')

        losses = list()
        accuracies = list()
        for batch in val_loader:
            x, y = batch
            x = x.to(device)
            y = y.to(device)

            # 1: forward
            with torch.no_grad():
                logit = model(x)

            # 2: compute the objective function
            J = loss(logit, y)

            losses.append(J.item())
            accuracies.append(y.eq(logit.detach().argmax(dim=1)).float().mean())

        loss_mean = torch.tensor(losses).mean()
        acc_mean = torch.tensor(accuracies).mean()

        if acc_mean > best_acc:
            best_acc = acc_mean
            PATH = f'{sample}/kfold_training/{k+1}/{ontology}'
            if not os.path.isdir(PATH):
                os.makedirs(PATH)
            torch.save(model, f'{PATH}/best_model_{best_acc:.4f}.pt')

        print(f'Epoch {epoch+1}, val loss: {loss_mean:.2f}, test accuracy: {acc_mean:.2f}')

    print(f'best accuracy obtained: {best_acc}')
    # end of epoch

Epoch 1, train loss: 3.07, train accuracy: 0.27
Epoch 1, val loss: 2.32, test accuracy: 0.32
Epoch 2, train loss: 2.23, train accuracy: 0.35
Epoch 2, val loss: 2.26, test accuracy: 0.34
Epoch 3, train loss: 2.09, train accuracy: 0.37
Epoch 3, val loss: 2.01, test accuracy: 0.38
Epoch 4, train loss: 1.96, train accuracy: 0.38
Epoch 4, val loss: 1.85, test accuracy: 0.40
Epoch 5, train loss: 1.83, train accuracy: 0.40
Epoch 5, val loss: 1.73, test accuracy: 0.42
Epoch 6, train loss: 1.73, train accuracy: 0.41
Epoch 6, val loss: 1.66, test accuracy: 0.41
Epoch 7, train loss: 1.66, train accuracy: 0.43
Epoch 7, val loss: 1.58, test accuracy: 0.46
Epoch 8, train loss: 1.60, train accuracy: 0.45
Epoch 8, val loss: 1.61, test accuracy: 0.43
Epoch 9, train loss: 1.57, train accuracy: 0.45
Epoch 9, val loss: 1.62, test accuracy: 0.44
Epoch 10, train loss: 1.53, train accuracy: 0.47
Epoch 10, val loss: 1.49, test accuracy: 0.47
Epoch 11, train loss: 1.49, train accuracy: 0.48
Epoch 11, val loss:

Epoch 87, val loss: 0.95, test accuracy: 0.61
Epoch 88, train loss: 1.05, train accuracy: 0.60
Epoch 88, val loss: 0.91, test accuracy: 0.65
Epoch 89, train loss: 1.04, train accuracy: 0.60
Epoch 89, val loss: 1.76, test accuracy: 0.46
Epoch 90, train loss: 1.02, train accuracy: 0.60
Epoch 90, val loss: 0.97, test accuracy: 0.63
Epoch 91, train loss: 1.02, train accuracy: 0.60
Epoch 91, val loss: 0.93, test accuracy: 0.64
Epoch 92, train loss: 1.02, train accuracy: 0.61
Epoch 92, val loss: 0.99, test accuracy: 0.62
Epoch 93, train loss: 1.03, train accuracy: 0.60
Epoch 93, val loss: 1.05, test accuracy: 0.60
Epoch 94, train loss: 1.02, train accuracy: 0.61
Epoch 94, val loss: 0.98, test accuracy: 0.63
Epoch 95, train loss: 1.02, train accuracy: 0.61
Epoch 95, val loss: 0.96, test accuracy: 0.62
Epoch 96, train loss: 1.03, train accuracy: 0.61
Epoch 96, val loss: 1.02, test accuracy: 0.61
Epoch 97, train loss: 0.99, train accuracy: 0.61
Epoch 97, val loss: 1.01, test accuracy: 0.60
Epoc

Epoch 172, val loss: 0.99, test accuracy: 0.62
Epoch 173, train loss: 0.93, train accuracy: 0.64
Epoch 173, val loss: 0.79, test accuracy: 0.69
Epoch 174, train loss: 0.91, train accuracy: 0.64
Epoch 174, val loss: 0.83, test accuracy: 0.66
Epoch 175, train loss: 0.95, train accuracy: 0.63
Epoch 175, val loss: 0.83, test accuracy: 0.66
Epoch 176, train loss: 0.92, train accuracy: 0.64
Epoch 176, val loss: 1.15, test accuracy: 0.55
Epoch 177, train loss: 0.89, train accuracy: 0.64
Epoch 177, val loss: 0.82, test accuracy: 0.68
Epoch 178, train loss: 0.92, train accuracy: 0.64
Epoch 178, val loss: 0.94, test accuracy: 0.61
Epoch 179, train loss: 0.91, train accuracy: 0.64
Epoch 179, val loss: 0.81, test accuracy: 0.67
Epoch 180, train loss: 0.89, train accuracy: 0.65
Epoch 180, val loss: 0.84, test accuracy: 0.66
Epoch 181, train loss: 0.91, train accuracy: 0.64
Epoch 181, val loss: 0.81, test accuracy: 0.68
Epoch 182, train loss: 0.96, train accuracy: 0.64
Epoch 182, val loss: 0.83, tes

Epoch 7, train loss: 1.71, train accuracy: 0.42
Epoch 7, val loss: 1.62, test accuracy: 0.45
Epoch 8, train loss: 1.64, train accuracy: 0.44
Epoch 8, val loss: 1.60, test accuracy: 0.43
Epoch 9, train loss: 1.61, train accuracy: 0.44
Epoch 9, val loss: 1.56, test accuracy: 0.46
Epoch 10, train loss: 1.57, train accuracy: 0.45
Epoch 10, val loss: 1.64, test accuracy: 0.44
Epoch 11, train loss: 1.60, train accuracy: 0.44
Epoch 11, val loss: 1.50, test accuracy: 0.47
Epoch 12, train loss: 1.53, train accuracy: 0.45
Epoch 12, val loss: 1.58, test accuracy: 0.45
Epoch 13, train loss: 1.50, train accuracy: 0.46
Epoch 13, val loss: 1.46, test accuracy: 0.48
Epoch 14, train loss: 1.56, train accuracy: 0.45
Epoch 14, val loss: 1.53, test accuracy: 0.46
Epoch 15, train loss: 1.49, train accuracy: 0.47
Epoch 15, val loss: 1.45, test accuracy: 0.48
Epoch 16, train loss: 1.46, train accuracy: 0.48
Epoch 16, val loss: 1.44, test accuracy: 0.50
Epoch 17, train loss: 1.42, train accuracy: 0.49
Epoch 1

Epoch 93, val loss: 0.90, test accuracy: 0.65
Epoch 94, train loss: 1.04, train accuracy: 0.60
Epoch 94, val loss: 0.94, test accuracy: 0.64
Epoch 95, train loss: 1.05, train accuracy: 0.60
Epoch 95, val loss: 0.96, test accuracy: 0.64
Epoch 96, train loss: 1.02, train accuracy: 0.60
Epoch 96, val loss: 1.00, test accuracy: 0.60
Epoch 97, train loss: 1.04, train accuracy: 0.60
Epoch 97, val loss: 0.98, test accuracy: 0.62
Epoch 98, train loss: 1.01, train accuracy: 0.61
Epoch 98, val loss: 1.00, test accuracy: 0.61
Epoch 99, train loss: 1.12, train accuracy: 0.58
Epoch 99, val loss: 0.98, test accuracy: 0.62
Epoch 100, train loss: 1.08, train accuracy: 0.60
Epoch 100, val loss: 0.95, test accuracy: 0.64
Epoch 101, train loss: 1.02, train accuracy: 0.61
Epoch 101, val loss: 0.90, test accuracy: 0.65
Epoch 102, train loss: 1.01, train accuracy: 0.60
Epoch 102, val loss: 1.05, test accuracy: 0.60
Epoch 103, train loss: 1.03, train accuracy: 0.61
Epoch 103, val loss: 0.92, test accuracy: 0

...

Epoch 105, val loss: 0.89, test accuracy: 0.64
Epoch 106, train loss: 0.99, train accuracy: 0.62
Epoch 106, val loss: 1.14, test accuracy: 0.58
Epoch 107, train loss: 0.97, train accuracy: 0.62
Epoch 107, val loss: 0.97, test accuracy: 0.62
Epoch 108, train loss: 0.98, train accuracy: 0.62
Epoch 108, val loss: 0.88, test accuracy: 0.65
Epoch 109, train loss: 0.99, train accuracy: 0.62
Epoch 109, val loss: 1.00, test accuracy: 0.62
Epoch 110, train loss: 1.01, train accuracy: 0.61
Epoch 110, val loss: 0.85, test accuracy: 0.67
Epoch 111, train loss: 0.96, train accuracy: 0.62
Epoch 111, val loss: 0.86, test accuracy: 0.66
Epoch 112, train loss: 0.98, train accuracy: 0.62
Epoch 112, val loss: 0.91, test accuracy: 0.64
Epoch 113, train loss: 0.97, train accuracy: 0.62
Epoch 113, val loss: 1.18, test accuracy: 0.57
Epoch 114, train loss: 0.98, train accuracy: 0.62
Epoch 114, val loss: 0.98, test accuracy: 0.60
Epoch 115, train loss: 0.97, train accuracy: 0.62
Epoch 115, val loss: 2.31, tes

Epoch 190, train loss: 0.91, train accuracy: 0.64
Epoch 190, val loss: 0.79, test accuracy: 0.69
Epoch 191, train loss: 0.88, train accuracy: 0.65
Epoch 191, val loss: 0.76, test accuracy: 0.69
Epoch 192, train loss: 0.88, train accuracy: 0.65
Epoch 192, val loss: 0.81, test accuracy: 0.67
Epoch 193, train loss: 0.93, train accuracy: 0.64
Epoch 193, val loss: 0.79, test accuracy: 0.67
Epoch 194, train loss: 0.87, train accuracy: 0.65
Epoch 194, val loss: 0.78, test accuracy: 0.69
Epoch 195, train loss: 0.89, train accuracy: 0.65
Epoch 195, val loss: 0.82, test accuracy: 0.67
Epoch 196, train loss: 0.88, train accuracy: 0.65
Epoch 196, val loss: 0.76, test accuracy: 0.69
Epoch 197, train loss: 0.90, train accuracy: 0.65
Epoch 197, val loss: 1.30, test accuracy: 0.54
Epoch 198, train loss: 0.89, train accuracy: 0.65
Epoch 198, val loss: 1.11, test accuracy: 0.57
Epoch 199, train loss: 0.87, train accuracy: 0.65
Epoch 199, val loss: 0.81, test accuracy: 0.67
Epoch 200, train loss: 0.86, t

Epoch 25, train loss: 1.24, train accuracy: 0.54
Epoch 25, val loss: 1.33, test accuracy: 0.52
Epoch 26, train loss: 1.24, train accuracy: 0.55
Epoch 26, val loss: 1.27, test accuracy: 0.53
Epoch 27, train loss: 1.22, train accuracy: 0.55
Epoch 27, val loss: 1.27, test accuracy: 0.53
Epoch 28, train loss: 1.23, train accuracy: 0.55
Epoch 28, val loss: 1.26, test accuracy: 0.55
Epoch 29, train loss: 1.20, train accuracy: 0.56
Epoch 29, val loss: 1.16, test accuracy: 0.56
Epoch 30, train loss: 1.20, train accuracy: 0.56
Epoch 30, val loss: 1.28, test accuracy: 0.56
Epoch 31, train loss: 1.20, train accuracy: 0.56
Epoch 31, val loss: 1.12, test accuracy: 0.59
Epoch 32, train loss: 1.18, train accuracy: 0.56
Epoch 32, val loss: 1.17, test accuracy: 0.55
Epoch 33, train loss: 1.19, train accuracy: 0.56
Epoch 33, val loss: 1.29, test accuracy: 0.52
Epoch 34, train loss: 1.19, train accuracy: 0.56
Epoch 34, val loss: 1.16, test accuracy: 0.56
Epoch 35, train loss: 1.17, train accuracy: 0.57
E

Epoch 111, val loss: 0.88, test accuracy: 0.65
Epoch 112, train loss: 0.98, train accuracy: 0.62
Epoch 112, val loss: 0.89, test accuracy: 0.64
Epoch 113, train loss: 0.99, train accuracy: 0.62
Epoch 113, val loss: 0.84, test accuracy: 0.67
Epoch 114, train loss: 0.96, train accuracy: 0.62
Epoch 114, val loss: 0.97, test accuracy: 0.61
Epoch 115, train loss: 1.01, train accuracy: 0.62
Epoch 115, val loss: 0.99, test accuracy: 0.59
Epoch 116, train loss: 0.97, train accuracy: 0.62
Epoch 116, val loss: 0.95, test accuracy: 0.63
Epoch 117, train loss: 1.01, train accuracy: 0.62
Epoch 117, val loss: 0.88, test accuracy: 0.65
Epoch 118, train loss: 0.97, train accuracy: 0.63
Epoch 118, val loss: 1.56, test accuracy: 0.50
Epoch 119, train loss: 0.99, train accuracy: 0.62
Epoch 119, val loss: 0.93, test accuracy: 0.64
Epoch 120, train loss: 0.98, train accuracy: 0.62
Epoch 120, val loss: 0.87, test accuracy: 0.65
Epoch 121, train loss: 0.97, train accuracy: 0.63
Epoch 121, val loss: 0.83, tes

Epoch 196, train loss: 0.90, train accuracy: 0.65
Epoch 196, val loss: 0.82, test accuracy: 0.68
Epoch 197, train loss: 0.96, train accuracy: 0.64
Epoch 197, val loss: 1.16, test accuracy: 0.55
Epoch 198, train loss: 0.90, train accuracy: 0.65
Epoch 198, val loss: 0.77, test accuracy: 0.70
Epoch 199, train loss: 0.86, train accuracy: 0.65
Epoch 199, val loss: 0.82, test accuracy: 0.66
Epoch 200, train loss: 0.93, train accuracy: 0.64
Epoch 200, val loss: 0.73, test accuracy: 0.72
Epoch 201, train loss: 0.88, train accuracy: 0.65
Epoch 201, val loss: 0.79, test accuracy: 0.68
Epoch 202, train loss: 0.87, train accuracy: 0.65
Epoch 202, val loss: 0.87, test accuracy: 0.64
Epoch 203, train loss: 0.89, train accuracy: 0.65
Epoch 203, val loss: 0.73, test accuracy: 0.72
Epoch 204, train loss: 0.89, train accuracy: 0.65
Epoch 204, val loss: 0.81, test accuracy: 0.67
Epoch 205, train loss: 0.88, train accuracy: 0.65
Epoch 205, val loss: 0.85, test accuracy: 0.66
Epoch 206, train loss: 0.92, t